# Classifying Urban sounds using Deep Learning

## 3 Model Training and Evaluation 

### Load Preprocessed data 

In [1]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

### Initial model architecture - MLP

We will start with constructing a Multilayer Perceptron (MLP) Neural Network using Keras and a Tensorflow backend. 

Starting with a `sequential` model so we can build the model layer by layer. 

We will begin with a simple model architecture, consisting of three layers, an input layer, a hidden layer and an output layer. All three layers will be of the `dense` layer type which is a standard layer type that is used in many cases for neural networks. 

The first layer will receive the input shape. As each sample contains 40 MFCCs (or columns) we have a shape of (1x40) this means we will start with an input shape of 40. 

The first two layers will have 256 nodes. The activation function we will be using for our first 2 layers is the `ReLU`, or `Rectified Linear Activation`. This activation function has been proven to work well in neural networks.

We will also apply a `Dropout` value of 50% on our first two layers. This will randomly exclude nodes from each update cycle which in turn results in a network that is capable of better generalisation and is less likely to overfit the training data.

Our output layer will have 10 nodes (num_labels) which matches the number of possible classifications. The activation is for our output layer is `softmax`. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has the highest probability.

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

Using TensorFlow backend.


### Compiling the model 

For compiling our model, we will use the following three parameters: 

* Loss function - we will use `categorical_crossentropy`. This is the most common choice for classification. A lower score indicates that the model is performing better.

* Metrics - we will use the `accuracy` metric which will allow us to view the accuracy score on the validation data when we train the model. 

* Optimizer - here we will use `adam` which is a generally good optimizer for many use cases.


In [3]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [4]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

### Training 

Here we will train the model. 

We will start with 100 epochs which is the number of times the model will cycle through the data. The model will improve on each cycle until it reaches a certain point. 

We will also start with a low batch size, as having a large batch size can reduce the generalisation ability of the model. 

In [5]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/100
6985/6985 [==============================] - 1s 108us/step - loss: 9.7211 - accuracy: 0.1791 - val_loss: 2.1744 - val_accuracy: 0.1660

Epoch 00001: val_loss improved from inf to 2.17442, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
6985/6985 [==============================] - 0s 71us/step - loss: 2.2919 - accuracy: 0.2331 - val_loss: 2.0901 - val_accuracy: 0.2347

Epoch 00002: val_loss improved from 2.17442 to 2.09008, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
6985/6985 [==============================] - 0s 70us/step - loss: 2.0790 - accuracy: 0.2753 - val_loss: 1.9319 - val_accuracy: 0.3251

Epoch 00003: val_loss improved from 2.09008 to 1.93192, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
6985/6985 [==============================] - 1s 76us/step - loss: 1.9199 - accuracy: 0.3220 - val_loss: 1.7101 - val_accuracy: 0.4362

Epoch 00004: val_loss improv

6985/6985 [==============================] - 0s 61us/step - loss: 0.7417 - accuracy: 0.7493 - val_loss: 0.5600 - val_accuracy: 0.8340

Epoch 00033: val_loss improved from 0.56754 to 0.56000, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 34/100
6985/6985 [==============================] - 0s 62us/step - loss: 0.7510 - accuracy: 0.7508 - val_loss: 0.5706 - val_accuracy: 0.8374

Epoch 00034: val_loss did not improve from 0.56000
Epoch 35/100
6985/6985 [==============================] - 0s 62us/step - loss: 0.7326 - accuracy: 0.7505 - val_loss: 0.5515 - val_accuracy: 0.8340

Epoch 00035: val_loss improved from 0.56000 to 0.55147, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 36/100
6985/6985 [==============================] - 0s 62us/step - loss: 0.6985 - accuracy: 0.7688 - val_loss: 0.5379 - val_accuracy: 0.8351

Epoch 00036: val_loss improved from 0.55147 to 0.53790, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 37/100
6985/6985 [=======

6985/6985 [==============================] - 1s 72us/step - loss: 0.5972 - accuracy: 0.8024 - val_loss: 0.4569 - val_accuracy: 0.8672

Epoch 00070: val_loss did not improve from 0.43216
Epoch 71/100
6985/6985 [==============================] - 0s 64us/step - loss: 0.5389 - accuracy: 0.8199 - val_loss: 0.4217 - val_accuracy: 0.8729

Epoch 00071: val_loss improved from 0.43216 to 0.42167, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 72/100
6985/6985 [==============================] - 0s 67us/step - loss: 0.5740 - accuracy: 0.8050 - val_loss: 0.4329 - val_accuracy: 0.8706

Epoch 00072: val_loss did not improve from 0.42167
Epoch 73/100
6985/6985 [==============================] - 0s 64us/step - loss: 0.5546 - accuracy: 0.8105 - val_loss: 0.4228 - val_accuracy: 0.8706

Epoch 00073: val_loss did not improve from 0.42167
Epoch 74/100
6985/6985 [==============================] - 0s 65us/step - loss: 0.5733 - accuracy: 0.8123 - val_loss: 0.4390 - val_accuracy: 0.8695

Epoch 0

### Test the model 

Here we will review the accuracy of the model on both the training and test data sets. 

In [6]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

('Training Accuracy: ', 0.9325698018074036)
('Testing Accuracy: ', 0.8815111517906189)


The initial Training and Testing accuracy scores are quite high. As there is not a great difference between the Training and Test scores (~5%) this suggests that the model has not suffered from overfitting. 

### Predictions  

Here we will build a method which will allow us to test the models predictions on a specified audio .wav file. 

In [7]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


/usr/local/lib/python2.7/site-packages/librosa/__init__.py:35: FutureWarning: You are using librosa with Python 2. Please note that librosa 0.7 will be the last version to support Python 2, after which it will require Python 3 or later.
  FutureWarning)


In [8]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

### Validation 

#### Test with sample data 

Initial sainity check to verify the predictions using a subsection of the sample audio files we explored in the first notebook. We expect the bulk of these to be classified correctly. 

In [9]:
# Class: Air Conditioner

filename = '../UrbanSound Dataset sample/audio/100852-0-0-0.wav' 
print_prediction(filename) 

('The predicted class is:', 'air_conditioner', '\n')
('air_conditioner', '\t\t : ', '0.99987435340881347656250000000000')
('car_horn', '\t\t : ', '0.00000180377708147716475650668144')
('children_playing', '\t\t : ', '0.00004441173950908705592155456543')
('dog_bark', '\t\t : ', '0.00000082766484865715028718113899')
('drilling', '\t\t : ', '0.00000444962461187969893217086792')
('engine_idling', '\t\t : ', '0.00000380217124984483234584331512')
('gun_shot', '\t\t : ', '0.00000718933097232365980744361877')
('jackhammer', '\t\t : ', '0.00005644194970955140888690948486')
('siren', '\t\t : ', '0.00000003597979869596201751846820')
('street_music', '\t\t : ', '0.00000661843705529463477432727814')


In [10]:
# Class: Drilling

filename = '../UrbanSound Dataset sample/audio/103199-4-0-0.wav'
print_prediction(filename) 

('The predicted class is:', 'drilling', '\n')
('air_conditioner', '\t\t : ', '0.00000281466668639041017740964890')
('car_horn', '\t\t : ', '0.00367502658627927303314208984375')
('children_playing', '\t\t : ', '0.00088050920749083161354064941406')
('dog_bark', '\t\t : ', '0.00264153117313981056213378906250')
('drilling', '\t\t : ', '0.77817684412002563476562500000000')
('engine_idling', '\t\t : ', '0.00000059014598718931665644049644')
('gun_shot', '\t\t : ', '0.00024957620189525187015533447266')
('jackhammer', '\t\t : ', '0.00007464970258297398686408996582')
('siren', '\t\t : ', '0.00000842899044073419645428657532')
('street_music', '\t\t : ', '0.21429006755352020263671875000000')


In [11]:
# Class: Street music 

filename = '../UrbanSound Dataset sample/audio/101848-9-0-0.wav'
print_prediction(filename) 

('The predicted class is:', 'street_music', '\n')
('air_conditioner', '\t\t : ', '0.00199427572079002857208251953125')
('car_horn', '\t\t : ', '0.00052700744708999991416931152344')
('children_playing', '\t\t : ', '0.06915031373500823974609375000000')
('dog_bark', '\t\t : ', '0.00242103193886578083038330078125')
('drilling', '\t\t : ', '0.00176948937587440013885498046875')
('engine_idling', '\t\t : ', '0.00083544914377853274345397949219')
('gun_shot', '\t\t : ', '0.00002691468398552387952804565430')
('jackhammer', '\t\t : ', '0.01022099796682596206665039062500')
('siren', '\t\t : ', '0.00001894417800940573215484619141')
('street_music', '\t\t : ', '0.91303551197052001953125000000000')


In [12]:
# Class: Car Horn 

filename = '../UrbanSound Dataset sample/audio/100648-1-0-0.wav'
print_prediction(filename) 

('The predicted class is:', 'street_music', '\n')
('air_conditioner', '\t\t : ', '0.00202822987921535968780517578125')
('car_horn', '\t\t : ', '0.05170848220586776733398437500000')
('children_playing', '\t\t : ', '0.03772762790322303771972656250000')
('dog_bark', '\t\t : ', '0.12638345360755920410156250000000')
('drilling', '\t\t : ', '0.04452101141214370727539062500000')
('engine_idling', '\t\t : ', '0.00369621324352920055389404296875')
('gun_shot', '\t\t : ', '0.01064509339630603790283203125000')
('jackhammer', '\t\t : ', '0.07278062403202056884765625000000')
('siren', '\t\t : ', '0.00250415946356952190399169921875')
('street_music', '\t\t : ', '0.64800506830215454101562500000000')


#### Observations 

From this brief sanity check the model seems to predict well. One errror was observed whereby a car horn was incorrectly classifed as a dog bark. 

We can see from the per class confidence that this was quite a low score (43%). This allows follows our early observation that a dog bark and car horn are similar in spectral shape. 

### Other audio

Here we will use a sample of various copyright free sounds that we not part of either our test or training data to further validate our model. 

In [13]:
filename = '../Evaluation audio/dog_bark_1.wav'
print_prediction(filename) 

('The predicted class is:', 'dog_bark', '\n')
('air_conditioner', '\t\t : ', '0.00069987989263609051704406738281')
('car_horn', '\t\t : ', '0.01771452277898788452148437500000')
('children_playing', '\t\t : ', '0.04449341446161270141601562500000')
('dog_bark', '\t\t : ', '0.63472759723663330078125000000000')
('drilling', '\t\t : ', '0.01072952337563037872314453125000')
('engine_idling', '\t\t : ', '0.00052825349848717451095581054688')
('gun_shot', '\t\t : ', '0.11306311190128326416015625000000')
('jackhammer', '\t\t : ', '0.00003311067484901286661624908447')
('siren', '\t\t : ', '0.00471432926133275032043457031250')
('street_music', '\t\t : ', '0.17329624295234680175781250000000')


In [14]:
filename = '../Evaluation audio/drilling_1.wav'

print_prediction(filename) 

('The predicted class is:', 'drilling', '\n')
('air_conditioner', '\t\t : ', '0.19057962298393249511718750000000')
('car_horn', '\t\t : ', '0.00000329012777910975273698568344')
('children_playing', '\t\t : ', '0.00067159335594624280929565429688')
('dog_bark', '\t\t : ', '0.00006981044862186536192893981934')
('drilling', '\t\t : ', '0.72829580307006835937500000000000')
('engine_idling', '\t\t : ', '0.00002228439188911579549312591553')
('gun_shot', '\t\t : ', '0.00033834224450401961803436279297')
('jackhammer', '\t\t : ', '0.08001644164323806762695312500000')
('siren', '\t\t : ', '0.00000008378015792231963132508099')
('street_music', '\t\t : ', '0.00000264926734416803810745477676')


In [15]:
filename = '../Evaluation audio/gun_shot_1.wav'

print_prediction(filename) 

# sample data weighted towards gun shot - peak in the dog barking sample is simmilar in shape to the gun shot sample

('The predicted class is:', 'street_music', '\n')
('air_conditioner', '\t\t : ', '0.00372563092969357967376708984375')
('car_horn', '\t\t : ', '0.00054582871962338685989379882812')
('children_playing', '\t\t : ', '0.00005983090159134007990360260010')
('dog_bark', '\t\t : ', '0.35962244868278503417968750000000')
('drilling', '\t\t : ', '0.00042311305878683924674987792969')
('engine_idling', '\t\t : ', '0.00089722994016483426094055175781')
('gun_shot', '\t\t : ', '0.00489394413307309150695800781250')
('jackhammer', '\t\t : ', '0.00001109897584683494642376899719')
('siren', '\t\t : ', '0.00045832575415261089801788330078')
('street_music', '\t\t : ', '0.62936258316040039062500000000000')


In [16]:
filename = '../Evaluation audio/siren_1.wav'

print_prediction(filename) 

('The predicted class is:', 'siren', '\n')
('air_conditioner', '\t\t : ', '0.00000666058394926949404180049896')
('car_horn', '\t\t : ', '0.00060503865825012326240539550781')
('children_playing', '\t\t : ', '0.00893795210868120193481445312500')
('dog_bark', '\t\t : ', '0.42400994896888732910156250000000')
('drilling', '\t\t : ', '0.00676910346373915672302246093750')
('engine_idling', '\t\t : ', '0.00661222776398062705993652343750')
('gun_shot', '\t\t : ', '0.01959343068301677703857421875000')
('jackhammer', '\t\t : ', '0.00019135596812702715396881103516')
('siren', '\t\t : ', '0.53200972080230712890625000000000')
('street_music', '\t\t : ', '0.00126460788305848836898803710938')


In [17]:
# Smith & Wesson downloaded from the internet

filename = '../Evaluation audio/40_smith_wesson_8x.wav'

print_prediction(filename) 

('The predicted class is:', 'gun_shot', '\n')
('air_conditioner', '\t\t : ', '0.00000000000000001276360530120758')
('car_horn', '\t\t : ', '0.00000000000013115150512919143688')
('children_playing', '\t\t : ', '0.00000024316776148225471843034029')
('dog_bark', '\t\t : ', '0.00020291317196097224950790405273')
('drilling', '\t\t : ', '0.00000000000127906149213763420391')
('engine_idling', '\t\t : ', '0.00000000000015279855017976545772')
('gun_shot', '\t\t : ', '0.99979680776596069335937500000000')
('jackhammer', '\t\t : ', '0.00000000000000000000000000366184')
('siren', '\t\t : ', '0.00000000001489281094280148209918')
('street_music', '\t\t : ', '0.00000000000000003008123030376870')


In [18]:

filename = '../Evaluation audio/Fireworks_And_Crowd.wav'

print_prediction(filename) 

('The predicted class is:', 'children_playing', '\n')
('air_conditioner', '\t\t : ', '0.02176773920655250549316406250000')
('car_horn', '\t\t : ', '0.00204630708321928977966308593750')
('children_playing', '\t\t : ', '0.37266218662261962890625000000000')
('dog_bark', '\t\t : ', '0.27142825722694396972656250000000')
('drilling', '\t\t : ', '0.00984426680952310562133789062500')
('engine_idling', '\t\t : ', '0.04833244904875755310058593750000')
('gun_shot', '\t\t : ', '0.10294465720653533935546875000000')
('jackhammer', '\t\t : ', '0.00037200117367319762706756591797')
('siren', '\t\t : ', '0.06650924682617187500000000000000')
('street_music', '\t\t : ', '0.10409288108348846435546875000000')


#### Observations 

The performance of our initial model is satisfactorry and has generalised well, seeming to predict well when tested against new audio data. 

### *In the next notebook we will refine our model*